In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os

from sqlmodel import Session, select

from app import database

from app.models import Step, Service

<IPython.core.display.Javascript object>

# Start Deployment Command

What should start deployment do?

* Create a deployment model with started = now() and finished = None timestamps
* Look up the steps from last deployment and recreate them as "pending" with new deployment id
* If there are no last steps, create one new dummy step
* Start deployment task
* Mark the first step as "running"

# Finish Step

What should finish step do?

* If the currently running step != the finished step, mark the currently running step as pending again and create a new finished step
* If the currently running step == the finished step, mark the currently running step as finished and set the next pending step to "running"

# Finish Deployment

What should happen on finish deployment?

* If the deployment has failed, do nothing
* If the deployment was successful, remove all pending steps

# Find Last Successful Deployment

In [38]:
with Session(database.engine) as session:
    stmt = """
        select distinct(step.deployment_id)
          from step, deployment
            on step.deployment_id=deployment.id
         where
           deployment.service_id = 4
           and step.state != 'success'
         group by step.deployment_id
        having count(step.id) > 1
    """
    result = session.exec(stmt).all()

In [62]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = 3
           and step.deployment_id not in (
            select distinct(step.deployment_id)
              from step, deployment
                on step.deployment_id=deployment.id
             where deployment.service_id = 3
               and step.state != 'success'
             group by step.deployment_id
            having count(step.id) > 1
           )
         group by step.deployment_id
        having count(step.id) > 0
         order by deployment.created desc
    """
    deployment_id = session.exec(stmt).all()[0][0]
    result = session.exec(stmt).all()

In [56]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = 3
         group by step.deployment_id
        having count(step.id) > 0
    """
    deployment_id = session.exec(stmt).all()[0][0]

In [54]:
len(result)

2

In [63]:
deployment_id

2

In [6]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = :service_id
           and step.deployment_id not in (
            select distinct(step.deployment_id)
              from step, deployment
                on step.deployment_id=deployment.id
             where deployment.service_id = :service_id
               and step.state != 'success'
             group by step.deployment_id
            having count(step.id) > 1
           )
         group by step.deployment_id
        having count(step.id) > 0
         order by deployment.started desc
    """
    deployment_id = session.exec(stmt, params={"service_id": 3}).all()[0][0]

print(deployment_id)

IndexError: list index out of range

In [3]:
with Session(database.engine) as session:
    help(session.exec)


Help on method exec in module sqlmodel.orm.session:

exec(statement: Union[sqlmodel.sql.expression.Select, sqlmodel.sql.expression.SelectOfScalar, sqlmodel.sql.base.Executable[~_TSelectParam]], *, params: Union[Mapping[str, Any], Sequence[Mapping[str, Any]], NoneType] = None, execution_options: Mapping[str, Any] = immutabledict({}), bind_arguments: Optional[Mapping[str, Any]] = None, _parent_execute_state: Optional[Any] = None, _add_event: Optional[Any] = None, **kw: Any) -> Union[sqlmodel.engine.result.Result[~_TSelectParam], sqlmodel.engine.result.ScalarResult[~_TSelectParam]] method of sqlmodel.orm.session.Session instance



<IPython.core.display.Javascript object>

In [3]:
with Session(database.engine) as session:
    services = session.exec(select(Service)).all()

/Users/jochen/.virtualenvs/fd/lib/python3.10/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


<IPython.core.display.Javascript object>

In [9]:
services[0].data["steps"]

[{'name': 'Gathering Facts'},
 {'name': 'Include secret vars'},
 {'name': 'Add nodejs signing key - do not download if present'},
 {'name': 'Add nodejs repository'},
 {'name': 'Install required build packages'},
 {'name': 'Add a unix user with a bash shell'},
 {'name': 'Git checkout repository'},
 {'name': 'Create .env file'},
 {'name': 'Create .vault_password file'},
 {'name': 'Install fastdeploy frontend vue app dependencies'},
 {'name': 'Build fastdeploy frontend vue app'},
 {'name': 'Create virtualenv for python'},
 {'name': 'Upgrade pip prod venv'},
 {'name': 'Create virtualenv for ansible'},
 {'name': 'Upgrade pip ansible venv'},
 {'name': 'Install ansible'},
 {'name': 'Create directory for ansible callback plugins'},
 {'name': 'Copy json stdout callback into default plugin directory'},
 {'name': 'Create sudoers file for deploy scripts'},
 {'name': 'Create directory for ansible inventory'},
 {'name': 'Create local inventory'},
 {'name': 'Create directory for server start script'}

<IPython.core.display.Javascript object>

In [10]:
services[1].data["steps"]

[{'name': 'message 1'},
 {'name': 'message 2'},
 {'name': 'message 3'},
 {'name': 'message 4'},
 {'name': 'message 5'},
 {'name': 'message 6'},
 {'name': 'message 7'},
 {'name': 'message 8'},
 {'name': 'message 9'}]

<IPython.core.display.Javascript object>

In [15]:
service = services[1]

<IPython.core.display.Javascript object>

In [16]:
await service.get_steps()

[StepBase(name='message 1', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 2', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 3', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 4', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 5', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 6', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 7', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 8', started=None, finished=None, state='pending', message=''),
 StepBase(name='message 9', started=None, finished=None, state='pending', message='')]

<IPython.core.display.Javascript object>

In [17]:
list(range(3))

[0, 1, 2]

<IPython.core.display.Javascript object>